## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-29-04-20-05 +0000,nyt,Trump and Schumer Move Toward Possible Deal to...,https://www.nytimes.com/2026/01/28/us/trump-sc...
1,2026-01-29-04-19-34 +0000,bbc,China executes 11 members of Myanmar scam mafia,https://www.bbc.com/news/articles/cx2gdrvy9gjo...
2,2026-01-29-04-12-27 +0000,nypost,"Massive inferno, explosions rip through SoCal ...",https://nypost.com/2026/01/28/us-news/massive-...
3,2026-01-29-04-09-14 +0000,nyt,Another Top Kennedy Center Official Resigns,https://www.nytimes.com/2026/01/28/us/politics...
4,2026-01-29-04-00-00 +0000,wsj,How Trump’s Push for Peace in Ukraine Will Unf...,https://www.wsj.com/world/russia/how-trumps-pu...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_3026/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,50
68,minneapolis,21
41,iran,19
178,ice,19
152,new,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
30,2026-01-29-03-00-00 +0000,wsj,Fed Chair Jerome Powell’s decision on whether ...,https://www.wsj.com/economy/central-banking/je...,136
46,2026-01-29-02-07-32 +0000,nyt,The Trump administration moves to sell a landm...,https://www.nytimes.com/live/2026/01/28/us/tru...,129
143,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...,125
226,2026-01-28-18-58-42 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...,123
134,2026-01-28-22-46-06 +0000,nyt,‘No Win’ for Minneapolis Police Caught Between...,https://www.nytimes.com/2026/01/28/us/politics...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
30,136,2026-01-29-03-00-00 +0000,wsj,Fed Chair Jerome Powell’s decision on whether ...,https://www.wsj.com/economy/central-banking/je...
208,86,2026-01-28-19-46-10 +0000,cbc,2 federal agents involved in shooting of Alex ...,https://www.cbc.ca/news/world/agents-pretti-sh...
226,73,2026-01-28-18-58-42 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...
143,64,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...
70,61,2026-01-29-01-00-00 +0000,wsj,Minneapolis Police and Residents Forge New Com...,https://www.wsj.com/us-news/minneapolis-police...
399,51,2026-01-28-06-26-29 +0000,nypost,South Korea’s former first lady Kim Keon Hee s...,https://nypost.com/2026/01/28/world-news/south...
384,48,2026-01-28-09-00-00 +0000,cbc,This Albertan signed up to fight in Ukraine. H...,https://www.cbc.ca/news/world/ukraine-david-ra...
253,47,2026-01-28-17-30-00 +0000,wsj,The Bank of Canada kept its policy rate unchan...,https://www.wsj.com/articles/bank-of-canada-st...
289,45,2026-01-28-15-27-40 +0000,bbc,"China trip will bring benefits to UK, Starmer ...",https://www.bbc.com/news/articles/c4gw47prew7o...
142,40,2026-01-28-22-26-00 +0000,wsj,Senate Democrats publicly laid out their deman...,https://www.wsj.com/politics/policy/senate-dem...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
